# Original Objective: Minimize total dose applied

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col = 0)

# create instance of solver
solver = pywraplp.Solver('Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_dosage_bounds = [0 for i in range(len(unspecified_voxels))]

# rectum voxels
rectum_voxels = [248,249,250,251,252,253,254,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313,314]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [103,104,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                    223,224,225,243,244]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [117,118,119,137,138,139,157,158,159,177,178,179,197,198,199,217,218,219,237,238,239,
                     257,258,259]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,175,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234,235]
CTV_dosage_bounds = [0 for i in range(len(CTV_voxels))]



############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(0, solver.infinity(), 'beamlet_'+str(i+1))

# list of all 400 voxel doses
voxel_dosages = [0 for i in range(400)]
for i in range(400):
    voxel_dosages[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))
    
# list to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)    
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i+=1



################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()
for i in range(400):
    objective.SetCoefficient(voxel_dosages[i], 1)
    

    
################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dosage
intensity_to_dosage_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dosage_linking[voxel] = solver.Constraint(0,0)
    intensity_to_dosage_linking[voxel].SetCoefficient(voxel_dosages[voxel], -1)
    for beamlet in range(60):
        intensity_to_dosage_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

        
# unspecified voxel constraints (voxels labeled starting at 1, not Python-style indexing)
for voxel_ID in unspecified_voxels:
    unspecified_dosage_bounds = solver.Constraint(0,72) # max dosage per unspecified voxel <= 72
    unspecified_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)

    
# rectum voxel constraints
rectum_dosage_bounds = [0 for i in range(len(rectum_voxels))]
for voxel_ID in rectum_voxels:
    rectum_dosage_bounds = solver.Constraint(0,79.2) # max dosage per rectum voxel <= 79.2
    rectum_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    
rectum_dosage_average = solver.Constraint(0,40) # average dosage per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    rectum_dosage_average.SetCoefficient(voxel_dosages[voxel_ID-1], 1/rectum_voxel_count)
    

# bladder voxel constraints
bladder_dosage_bounds = [0 for i in range(bladder_voxel_count)]
for voxel_ID in bladder_voxels:
    bladder_dosage_bounds = solver.Constraint(0,81) # max dosage per bladder voxel <= 81
    bladder_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    
bladder_dosage_average = solver.Constraint(0,50) # average dosage per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    bladder_dosage_average.SetCoefficient(voxel_dosages[voxel_ID-1], 1/bladder_voxel_count)

# linking voxel_dosages to bladder_above_threshold to fill 1/0 based on dose
# i = 0
# for voxel_ID in bladder_voxels:
#     bladder_threshold = solver.Constraint(-solver.infinity(),65)
#     bladder_threshold.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
#     bladder_threshold.SetCoefficient(bladder_above_threshold[i], -100)
#     i+=1
i = 0
for voxel_ID in bladder_voxels: # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold = solver.Constraint(0,solver.infinity())
    bladder_threshold.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    bladder_threshold.SetCoefficient(bladder_above_threshold[i], -65)
    i+=1
j = 0
for voxel_ID in bladder_voxels: # this forces to be 1 if greater than 65
    bladder_threshold = solver.Constraint(-solver.infinity(),65)
    bladder_threshold.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    bladder_threshold.SetCoefficient(bladder_above_threshold[j], -100)
    j+=1
    
# bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have dosages > 65
# bladder_percentage_lower = solver.Constraint(0,bladder_percentage_count) 
# for binary in bladder_above_threshold:
#     bladder_percentage_lower.SetCoefficient(binary, 1)


# left femur voxel constraints
left_femur_dosage_bounds = [0 for i in range(left_femur_voxel_count)]
for voxel_ID in left_femur_voxels:
    left_femur_dosage_bounds = solver.Constraint(0,50) # max dosage per left femur voxel <= 50
    left_femur_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    
# need percentage constraint here, max 15% >40
    
    
# right femur voxel constraints
right_femur_dosage_bounds = [0 for i in range(right_femur_voxel_count)]
for voxel_ID in right_femur_voxels:
    right_femur_dosage_bounds = solver.Constraint(0,50) # max dosage per right femur voxel <= 50
    right_femur_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)

# need percentage constraint here, max 15% >40


# CTV voxel constraints
for voxel_ID in CTV_voxels:
    #(80.73,84.87) # each voxel within +/-2.5% of target dosage (82.8) this is ideal
    CTV_dosage_bounds = solver.Constraint(60,85)
    CTV_dosage_bounds.SetCoefficient(voxel_dosages[voxel_ID-1], 1)
    


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()

# Put results into lists to be able to work with them for metrics (avoid OR-Tools-related errors)
dosage_plan = []
for i in voxel_dosages:
    dosage_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())
    
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

# left_femur_threshold = []
# for i in left_femur_above_threshold:
#     left_femur_threshold.append(i.solution_value())
    
# right_femur_threshold = []
# for i in right_femur_above_threshold:
#     right_femur_threshold.append(i.solution_value())

# Total Dosage
# print("Total Dosage", objective.Value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# dosage_matrix = np.reshape(voxel_dosages, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = dosage_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Dosages
print("Unspecified:")
for voxel_ID in unspecified_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")
print("Rectum:")
for voxel_ID in rectum_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")
print("Bladder:")
for voxel_ID in bladder_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")
print("Left Femur:")
for voxel_ID in left_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")
print("Right Femur:")
for voxel_ID in right_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")
print("CTV:")
for voxel_ID in CTV_voxels:
    print("Voxel_"+str(voxel_ID), voxel_dosages[voxel_ID-1].solution_value())
print("\n")


# Beamlet Intensities
# for i in range(len(beamlet_intensities)):
#     print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


# Metric Computation Cell

In [ ]:
################################################### Metrics ################################################

# added .1 to all limits for tolerance on x.0001-type cases

### unspecified ###
# max dose metric
unspecified_max_excess = []
for voxel_ID in unspecified_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>72):
        unspecified_max_excess.append(dose_value)
# print("List of Unspecified Doses >72: ", unspecified_max_excess)
print("# of Unspecified Doses >72 (0): ", len(unspecified_max_excess))
unspecified_max_overshoot = sum(unspecified_overshoot_list)
print("Unspecified Max Error: ", unspecified_max_overshoot, "\n\n")
    

### rectum ###
# max dose metric
rectum_max_excess = []
for voxel_ID in rectum_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>79.21):
        rectum_max_excess.append(dose_value)
# print("List of Rectum Doses >79.2: ", rectum_max_excess)
print("# of Rectum Doses >79.2 (0): ", len(rectum_max_excess))
# average dose metric
rectum_total_dose = 0
for voxel_ID in rectum_voxels:
    rectum_total_dose += dose_plan[voxel_ID-1]
print("Average Rectum Dose (max 40): ", rectum_total_dose/rectum_voxel_count)
rectum_max_overshoot = sum(rectum_overshoot_list)
print("Rectum Max Error: ", rectum_max_overshoot)
print("Rectum Average Error: ", rectum_average_overshoot, "\n\n")


### bladder ###
# max dose metric
bladder_max_excess = []
for voxel_ID in bladder_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>81.1):
        bladder_max_excess.append(dose_value)
# print("List of Bladder Doses >81: ", bladder_max_excess)
print("# of Bladder Doses >81 (0): ", len(bladder_max_excess))
# average dose metric
bladder_total_dose = 0
for voxel_ID in bladder_voxels:
    bladder_total_dose += dose_plan[voxel_ID-1]
print("Average Bladder Dose (max 50): ", bladder_total_dose/bladder_voxel_count)
# percentage metric
bladder_percentage_excess = []
for voxel_ID in bladder_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>65.1):
        bladder_percentage_excess.append(dose_value)
# print("List of Bladder Doses >65: ", bladder_percentage_excess)
print("% of Bladder Doses >65 (max 10%): ", 100*len(bladder_percentage_excess)/bladder_voxel_count)
bladder_max_overshoot = sum(bladder_overshoot_list)
print("Bladder Max Error: ", bladder_max_overshoot)
print("Bladder Average Error: ", bladder_average_overshoot)
print("Bladder Percentage Error: ", bladder_percentage_overshoot, "\n\n")


### left femur ###
# max dose metric
left_femur_max_excess = []
for voxel_ID in left_femur_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>50.1):
        left_femur_max_excess.append(dose_value)
# print("List of Left Femur Doses >50: ", left_femur_max_excess)
print("# of Left Femur Doses >50 (0): ", len(left_femur_max_excess))
# percentage metric
left_femur_percentage_excess = []
for voxel_ID in left_femur_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>40.1):
        left_femur_percentage_excess.append(dose_value)
# print("List of Left Femur Doses >40: ", left_femur_percentage_excess)
print("% of Left Femur Doses >40 (max 15%): ", 100*len(left_femur_percentage_excess)/left_femur_voxel_count)
left_femur_max_overshoot = sum(left_femur_overshoot_list)
print("Left Femur Max Error: ", left_femur_max_overshoot)
print("Left Femur Percentage Error: ", left_femur_percentage_overshoot, "\n\n")


### right femur ###
# max dose metric
right_femur_max_excess = []
for voxel_ID in right_femur_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>50.1):
        right_femur_max_excess.append(dose_value)
# print("List of Right Femur Doses >50: ", right_femur_max_excess)
print("# of Right Femur Doses >50 (0): ", len(right_femur_max_excess))
# percentage metric
right_femur_percentage_excess = []
for voxel_ID in right_femur_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (dose_value>40.1):
        right_femur_percentage_excess.append(dose_value)
# print("List of Right Femur Doses >40: ", right_femur_percentage_excess)
print("% of Right Femur Doses >40 (max 15%): ", 100*len(right_femur_percentage_excess)/right_femur_voxel_count)
right_femur_max_overshoot = sum(right_femur_overshoot_list)
print("Right Femur Max Error: ", right_femur_max_overshoot)
print("Right Femur Percentage Error: ", right_femur_percentage_overshoot, "\n\n")


### CTV ###
# uniformity metric
CTV_deviation = []
for voxel_ID in CTV_voxels:
    dose_value = dose_plan[voxel_ID-1]
    if (abs(dose_value-82.8)>2.0701):
        CTV_deviation.append(dose_value)
print("List of CTV Doses with >2.5% Deviation from 82.8: ", CTV_deviation)
print("# of CTV Doses with >2.5% Deviation from 82.8 (0): ", len(CTV_deviation))
CTV_deviation = sum(CTV_overshoot_list) + sum(CTV_undershoot_list)
print("CTV Deviation Error: ", CTV_deviation)


# Base Program with New Objective: Minimize deviation from parameters

In [ ]:
# Objective: Minimize sum of deviation from given constraints (unweighted)

# next one: try weighting averages more heavily since higher average more impactful than higher single value


##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col = 0)

# create instance of solver
solver = pywraplp.Solver('Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)



############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))
    
    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)    
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i+=1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i+=1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i+=1
    

# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))
    
rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))

rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))

bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')

bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))

left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))

right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))

CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))
    
    

################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], 1)
    
for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, 1)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, 1)
objective.SetCoefficient(bladder_percentage_overshoot, 1)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 1)
objective.SetCoefficient(left_femur_percentage_overshoot, 1)
    
for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 1)
objective.SetCoefficient(right_femur_percentage_overshoot, 1)
    
for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 1)
    objective.SetCoefficient(CTV_undershoot[i], 1)
    
""" minimize sum of
-exceeding 72 for unspecified - DONE
-exceeding 79.2 for rectum - DONE
-average exceeding 40 for rectum - DONE
-exceeding 81 for bladder - DONE
-average exceeding 50 for bladder - DONE
-percent over 10% receiving dose over 65 for bladder - DONE
-exceeding 50 for left femur - DONE
-percent over 15% receiving dose over 40 for left femur - DONE
-exceeding 50 for right femur - DONE
-percent over 15% receiving dose over 40 for right femur - DONE
-exceeding 82.8 for CTV - DONE
-being below 82.8 for CTV - DONE
""" 
    

    
################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0,0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels: # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0,solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    i+=1
j = 0
for voxel_ID in bladder_voxels: # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(),65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[j], -100)
    j+=1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: # this forces to be 0 if less than 40, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0,solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    i+=1
j = 0
for voxel_ID in left_femur_voxels: # this forces to be 1 if greater than 40
    left_femur_threshold1 = solver.Constraint(-solver.infinity(),40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[j], -100)
    j+=1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels: # this forces to be 0 if less than 40, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0,solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    i+=1
j = 0
for voxel_ID in right_femur_voxels: # this forces to be 1 if greater than 40
    right_femur_threshold1 = solver.Constraint(-solver.infinity(),40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[j], -100)
    j+=1

        
# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    unspecified_dose_max = solver.Constraint(0,72) # max dose per unspecified voxel <= 72
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i+=1

    
# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
for voxel_ID in rectum_voxels:
    rectum_dose_max = solver.Constraint(0,79.2) # max dose per rectum voxel <= 79.2
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i+=1
    
rectum_dose_average = solver.Constraint(0,40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    rectum_dose_average.SetCoefficient(voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel_ID in bladder_voxels:
    bladder_dose_max = solver.Constraint(0,81) # max dose per bladder voxel <= 81
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i+=1
    
bladder_dose_average = solver.Constraint(0,50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    bladder_dose_average.SetCoefficient(voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0,bladder_percentage_count) 
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    left_femur_dose_max = solver.Constraint(0,50) # max dose per left femur voxel <= 50
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i+=1
    
left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0,left_femur_percentage_count) 
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(left_femur_percentage_overshoot, -left_femur_voxel_count/100)
    
    
# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    right_femur_dose_max = solver.Constraint(0,50) # max dose per right femur voxel <= 50
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i+=1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(0,right_femur_percentage_count) 
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    CTV_dose_target = solver.Constraint(82.8,82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i+=1
    


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())

rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())

bladder_average_overshoot = bladder_average_overshoot.solution_value()

bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
    
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
    
right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
    
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()

CTV_overshoot_list = []
for i in CTV_overshoot:
    CTV_overshoot_list.append(i.solution_value())

CTV_undershoot_list = []
for i in CTV_undershoot:
    CTV_undershoot_list.append(i.solution_value())
    
    
    
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())
    
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
print("Unspecified:")
for voxel_ID in unspecified_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Rectum:")
for voxel_ID in rectum_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Bladder:")
for voxel_ID in bladder_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Left Femur:")
for voxel_ID in left_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Right Femur:")
for voxel_ID in right_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("CTV:")
for voxel_ID in CTV_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")


# Beamlet Intensities
# for i in range(len(beamlet_intensities)):
#     print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


# Condensed version of base program (eliminated extraneous loops etc., original version kept for readability in editing and debugging)

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)

############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], 1)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, 1)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, 1)
objective.SetCoefficient(bladder_percentage_overshoot, 1)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 1)
objective.SetCoefficient(left_femur_percentage_overshoot, 1)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 1)
objective.SetCoefficient(right_femur_percentage_overshoot, 1)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 1)
    objective.SetCoefficient(CTV_undershoot[i], 1)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
print("Unspecified:")
for voxel_ID in unspecified_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Rectum:")
for voxel_ID in rectum_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Bladder:")
for voxel_ID in bladder_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Left Femur:")
for voxel_ID in left_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("Right Femur:")
for voxel_ID in right_femur_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")
print("CTV:")
for voxel_ID in CTV_voxels:
    print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
print("\n")


# Beamlet Intensities
# for i in range(len(beamlet_intensities)):
#     print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())

# Formulation of all 3 plans using original CTV constraint, record error when there is any deviation from 82.8 Gy

## Aggressive (no CTV error, low femur error, high error for other regions)

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], .4)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 3)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 3)
objective.SetCoefficient(left_femur_percentage_overshoot, 12)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 2)
objective.SetCoefficient(right_femur_percentage_overshoot, 3)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 81)
    objective.SetCoefficient(CTV_undershoot[i], 60)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    #CTV_dose_target = solver.Constraint(80.73, 84.87)
    CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


## Balanced (low CTV error, medium error for femur and other regions)

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], .5)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count*1.1)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 8)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 5)
objective.SetCoefficient(left_femur_percentage_overshoot, 12)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 1)
objective.SetCoefficient(right_femur_percentage_overshoot, 3)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 45)
    objective.SetCoefficient(CTV_undershoot[i], 45)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    #CTV_dose_target = solver.Constraint(80.73, 84.87)
    CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


## Relaxed (medium error for all regions)

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], .75)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count*1.4)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 12)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 2)
objective.SetCoefficient(left_femur_percentage_overshoot, 12)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 1)
objective.SetCoefficient(right_femur_percentage_overshoot, 2)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 25)
    objective.SetCoefficient(CTV_undershoot[i], 25)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    #CTV_dose_target = solver.Constraint(80.73, 84.87)
    CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


# Formulation of all 3 plans with revised CTV constraint, only record error when the deviation from 82.8 Gy is greater than 2.5%

## Aggressive

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], .5)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 3)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 5)
objective.SetCoefficient(left_femur_percentage_overshoot, 18)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 3)
objective.SetCoefficient(right_femur_percentage_overshoot, 5)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 100)
    objective.SetCoefficient(CTV_undershoot[i], 100)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    CTV_dose_target = solver.Constraint(80.73, 84.87)
    #CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


## Balanced

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], 1)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count*1.2)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 1.5)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 4)
objective.SetCoefficient(left_femur_percentage_overshoot, 18)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 2)
objective.SetCoefficient(right_femur_percentage_overshoot, 6)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 40)
    objective.SetCoefficient(CTV_undershoot[i], 40)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    CTV_dose_target = solver.Constraint(80.73, 84.87)
    #CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())


## Relaxed

In [ ]:
##################################################### SETUP #######################################################

import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

# import DoseMatrix file and load it as a DataFrame
intensity_to_dose = pd.read_excel('DoseMatrix.xlsx', index_col=0)

# create instance of solver
solver = pywraplp.Solver(
    'Project', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


############################################# VOXEL REGION ASSIGNMENT #############################################

# unspecified voxels
unspecified_voxels = []
unspecified_voxels.append([i for i in range(1,88)])
unspecified_voxels.append([i for i in range(95,103)])
unspecified_voxels.append([105,106,116,120,121,122,126,136,140,141,146,147,155,156,160,161,166,167,176,180,
                           181,186,196,200,201,206,216,220,221,222,226,227,236,240,241,242,245,246,247,255,256])
unspecified_voxels.append([i for i in range(260,268)])
unspecified_voxels.append([i for i in range(275,288)])
unspecified_voxels.append([i for i in range(295,309)])
unspecified_voxels.append([i for i in range(315,401)])
unspecified_voxels = [item for sublist in unspecified_voxels for item in sublist]
unspecified_voxel_count = len(unspecified_voxels)

# rectum voxels
rectum_voxels = [249,250,251,252,253,268,269,270,271,272,273,274,288,289,290,291,292,293,294,309,
                310,311,312,313]
rectum_voxel_count = len(rectum_voxels)

# bladder voxels
bladder_voxels = [88,89,90,91,92,93,94,107,108,109,110,111,112,113,114,115,127,128,129,130,131,132,133,134,
                 135,148,149,150,151,152,153,154]
bladder_voxel_count = len(bladder_voxels)

# left femur voxels
left_femur_voxels = [117,118,119,136,137,138,139,156,157,158,159,176,177,178,179,196,197,198,199,216,217,
                     218,219,236,237,238,239,257,258,259]
left_femur_voxel_count = len(left_femur_voxels)

# right femur voxels
right_femur_voxels = [103,104,122,123,124,125,142,143,144,145,162,163,164,165,182,183,184,185,202,203,204,205,
                      222,223,224,225,243,244]
right_femur_voxel_count = len(right_femur_voxels)

# CTV voxels
CTV_voxels = [168,169,170,171,172,173,174,187,188,189,190,191,192,193,194,195,207,208,209,210,211,212,
             213,214,215,228,229,230,231,232,233,234]
CTV_voxel_count = len(CTV_voxels)


############################################### DECISION VARIABLES ################################################

# list of all 60 beamlet intensities
beamlet_intensities = [0 for i in range(60)]
for i in range(60):
    beamlet_intensities[i] = solver.NumVar(
        0, solver.infinity(), 'beamlet_'+str(i+1))

    
# list of all 400 voxel doses
voxel_doses = [0 for i in range(400)]
for i in range(400):
    voxel_doses[i] = solver.NumVar(0, solver.infinity(), 'voxel_'+str(i+1))

    
# binary variables to track amount of bladder voxels receiving dose >65 (=1 if >65, 0 otherwise)
bladder_above_threshold = [0 for i in range(bladder_voxel_count)]
i = 0
for voxel in bladder_voxels:
    bladder_above_threshold[i] = solver.IntVar(0, 1, 'bladder_above_threshold_'+str(voxel))
    i += 1

left_femur_above_threshold = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel in left_femur_voxels:
    left_femur_above_threshold[i] = solver.IntVar(0, 1, 'left_femur_above_threshold_'+str(voxel))
    i += 1

right_femur_above_threshold = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel in right_femur_voxels:
    right_femur_above_threshold[i] = solver.IntVar(0, 1, 'right_femur_above_threshold_'+str(voxel))
    i += 1


# 'z' variables, measure distance from satisfying constraints
unspecified_overshoot = [0 for i in range(unspecified_voxel_count)]
for i in range(unspecified_voxel_count):
    unspecified_overshoot[i] = solver.NumVar(0, solver.infinity(), 'unspecified_overshoot_'+str(i))

rectum_overshoot = [0 for i in range(rectum_voxel_count)]
for i in range(rectum_voxel_count):
    rectum_overshoot[i] = solver.NumVar(0, solver.infinity(), 'rectum_overshoot_'+str(i))
rectum_average_overshoot = solver.NumVar(0, solver.infinity(), 'rectum_average_overshoot')

bladder_overshoot = [0 for i in range(bladder_voxel_count)]
for i in range(bladder_voxel_count):
    bladder_overshoot[i] = solver.NumVar(0, solver.infinity(), 'bladder_overshoot_'+str(i))
bladder_average_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_average_overshoot')
bladder_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'bladder_percentage_overshoot')

left_femur_overshoot = [0 for i in range(left_femur_voxel_count)]
for i in range(left_femur_voxel_count):
    left_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'left_femur_overshoot_'+str(i))
left_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'left_femur_percentage_overshoot')

right_femur_overshoot = [0 for i in range(right_femur_voxel_count)]
for i in range(right_femur_voxel_count):
    right_femur_overshoot[i] = solver.NumVar(0, solver.infinity(), 'right_femur_overshoot_'+str(i))
right_femur_percentage_overshoot = solver.NumVar(0, solver.infinity(), 'right_femur_percentage_overshoot')

CTV_overshoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_overshoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_overshoot_'+str(i))
CTV_undershoot = [0 for i in range(CTV_voxel_count)]
for i in range(CTV_voxel_count):
    CTV_undershoot[i] = solver.NumVar(0, solver.infinity(), 'CTV_undershoot_'+str(i))


################################################### OBJECTIVE ####################################################

objective = solver.Objective()
objective.SetMinimization()

for i in range(unspecified_voxel_count):
    objective.SetCoefficient(unspecified_overshoot[i], 1.5)

for i in range(rectum_voxel_count):
    objective.SetCoefficient(rectum_overshoot[i], 1)
objective.SetCoefficient(rectum_average_overshoot, rectum_voxel_count*1.5)

for i in range(bladder_voxel_count):
    objective.SetCoefficient(bladder_overshoot[i], 1)
objective.SetCoefficient(bladder_average_overshoot, bladder_voxel_count)
objective.SetCoefficient(bladder_percentage_overshoot, 2)

for i in range(left_femur_voxel_count):
    objective.SetCoefficient(left_femur_overshoot[i], 4)
objective.SetCoefficient(left_femur_percentage_overshoot, 18)

for i in range(right_femur_voxel_count):
    objective.SetCoefficient(right_femur_overshoot[i], 1)
objective.SetCoefficient(right_femur_percentage_overshoot, 4)

for i in range(CTV_voxel_count):
    objective.SetCoefficient(CTV_overshoot[i], 25)
    objective.SetCoefficient(CTV_undershoot[i], 25)


################################################## CONSTRAINTS ###################################################

# linking decision variables for beamlet intensity and voxel dose
intensity_to_dose_linking = [0 for i in range(400)]
for voxel in range(400):
    intensity_to_dose_linking[voxel] = solver.Constraint(0, 0)
    intensity_to_dose_linking[voxel].SetCoefficient(voxel_doses[voxel], -1)
    for beamlet in range(60):
        intensity_to_dose_linking[voxel].SetCoefficient(
            beamlet_intensities[beamlet], intensity_to_dose.iloc[voxel, beamlet])

# linking decision variables for voxel dose and binary bladder above threshold (1 if >65)
i = 0
for voxel_ID in bladder_voxels:  
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    bladder_threshold0 = solver.Constraint(0, solver.infinity())
    bladder_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold0.SetCoefficient(bladder_above_threshold[i], -65)
    # this forces to be 1 if greater than 65
    bladder_threshold1 = solver.Constraint(-solver.infinity(), 65)
    bladder_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_threshold1.SetCoefficient(bladder_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary left femur above threshold (1 if >40)
i = 0
for voxel_ID in left_femur_voxels: 
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    left_femur_threshold0 = solver.Constraint(0, solver.infinity())
    left_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold0.SetCoefficient(left_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    left_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    left_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_threshold1.SetCoefficient(left_femur_above_threshold[i], -100)
    i += 1

# linking decision variables for voxel dose and binary right femur above threshold (1 if >40)
i = 0
for voxel_ID in right_femur_voxels:
    # this forces to be 0 if less than 65, but doesn't force 1 if greater
    right_femur_threshold0 = solver.Constraint(0, solver.infinity())
    right_femur_threshold0.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold0.SetCoefficient(right_femur_above_threshold[i], -40)
    # this forces to be 1 if greater than 65
    right_femur_threshold1 = solver.Constraint(-solver.infinity(), 40)
    right_femur_threshold1.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_threshold1.SetCoefficient(right_femur_above_threshold[i], -100)
    i += 1


# unspecified voxel constraints
unspecified_dose_max = [0 for i in range(len(unspecified_voxels))]
i = 0
for voxel_ID in unspecified_voxels:
    # max dose per unspecified voxel <= 72
    unspecified_dose_max = solver.Constraint(0, 72)
    unspecified_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    unspecified_dose_max.SetCoefficient(unspecified_overshoot[i], -1)
    i += 1


# rectum voxel constraints
rectum_dose_max = [0 for i in range(len(rectum_voxels))]
i = 0
rectum_dose_average = solver.Constraint(0, 40) # average dose per rectum voxel <= 40
for voxel_ID in rectum_voxels:
    # max dose per rectum voxel <= 79.2
    rectum_dose_max = solver.Constraint(0, 79.2)
    rectum_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    rectum_dose_max.SetCoefficient(rectum_overshoot[i], -1)
    i += 1
    # average dose per rectum voxel <= 40
    rectum_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/rectum_voxel_count)
rectum_dose_average.SetCoefficient(rectum_average_overshoot, -1)


# bladder voxel constraints
bladder_dose_max = [0 for i in range(bladder_voxel_count)]
i = 0
bladder_dose_average = solver.Constraint(0, 50) # average dose per bladder voxel <= 50
for voxel_ID in bladder_voxels:
    # max dose per bladder voxel <= 81
    bladder_dose_max = solver.Constraint(0, 81)
    bladder_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    bladder_dose_max.SetCoefficient(bladder_overshoot[i], -1)
    i += 1
    # average dose per bladder voxel <= 50
    bladder_dose_average.SetCoefficient(
        voxel_doses[voxel_ID-1], 1/bladder_voxel_count)
bladder_dose_average.SetCoefficient(bladder_average_overshoot, -1)

bladder_percentage_count = .1*bladder_voxel_count # max 10% of bladder voxels have doses > 65
bladder_percentage_lower = solver.Constraint(0, bladder_percentage_count)
for binary in bladder_above_threshold:
    bladder_percentage_lower.SetCoefficient(binary, 1)
bladder_percentage_lower.SetCoefficient(
    bladder_percentage_overshoot, -bladder_voxel_count/100)


# left femur voxel constraints
left_femur_dose_max = [0 for i in range(left_femur_voxel_count)]
i = 0
for voxel_ID in left_femur_voxels:
    # max dose per left femur voxel <= 50
    left_femur_dose_max = solver.Constraint(0, 50)
    left_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    left_femur_dose_max.SetCoefficient(left_femur_overshoot[i], -1)
    i += 1

left_femur_percentage_count = .15*left_femur_voxel_count # max 15% of left femur voxels have doses >40
left_femur_percentage_lower = solver.Constraint(0, left_femur_percentage_count)
for binary in left_femur_above_threshold:
    left_femur_percentage_lower.SetCoefficient(binary, 1)
left_femur_percentage_lower.SetCoefficient(
    left_femur_percentage_overshoot, -left_femur_voxel_count/100)


# right femur voxel constraints
right_femur_dose_max = [0 for i in range(right_femur_voxel_count)]
i = 0
for voxel_ID in right_femur_voxels:
    # max dose per right femur voxel <= 50
    right_femur_dose_max = solver.Constraint(0, 50)
    right_femur_dose_max.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    right_femur_dose_max.SetCoefficient(right_femur_overshoot[i], -1)
    i += 1

right_femur_percentage_count = .15*right_femur_voxel_count # max 15% of right femur voxels have doses >40
right_femur_percentage_lower = solver.Constraint(
    0, right_femur_percentage_count)
for binary in right_femur_above_threshold:
    right_femur_percentage_lower.SetCoefficient(binary, 1)
right_femur_percentage_lower.SetCoefficient(
    right_femur_percentage_overshoot, -right_femur_voxel_count/100)


# CTV voxel constraints
CTV_dose_target = [0 for i in range(CTV_voxel_count)]
i = 0
for voxel_ID in CTV_voxels:
    CTV_dose_target = solver.Constraint(80.73, 84.87)
    #CTV_dose_target = solver.Constraint(82.8, 82.8)
    CTV_dose_target.SetCoefficient(voxel_doses[voxel_ID-1], 1)
    CTV_dose_target.SetCoefficient(CTV_overshoot[i], -1)
    CTV_dose_target.SetCoefficient(CTV_undershoot[i], 1)
    i += 1


########################################### SOLVER AND RESULT OUTPUT ############################################

solver.Solve()
print(objective.Value())

# Put results into lists to be able to work with them and avoid OR-Tools-related errors with decision variables
dose_plan = []
for i in voxel_doses:
    dose_plan.append(i.solution_value())

beamlet_plan = []
for i in beamlet_intensities:
    beamlet_plan.append(i.solution_value())

unspecified_overshoot_list = []
for i in unspecified_overshoot:
    unspecified_overshoot_list.append(i.solution_value())

rectum_overshoot_list = []
for i in rectum_overshoot:
    rectum_overshoot_list.append(i.solution_value())
rectum_average_overshoot = rectum_average_overshoot.solution_value()

bladder_overshoot_list = []
for i in bladder_overshoot:
    bladder_overshoot_list.append(i.solution_value())
bladder_average_overshoot = bladder_average_overshoot.solution_value()
bladder_percentage_overshoot = bladder_percentage_overshoot.solution_value()
bladder_threshold = []
for i in bladder_above_threshold:
    bladder_threshold.append(i.solution_value())

left_femur_overshoot_list = []
for i in left_femur_overshoot:
    left_femur_overshoot_list.append(i.solution_value())
left_femur_percentage_overshoot = left_femur_percentage_overshoot.solution_value()
left_femur_threshold = []
for i in left_femur_above_threshold:
    left_femur_threshold.append(i.solution_value())

right_femur_overshoot_list = []
for i in right_femur_overshoot:
    right_femur_overshoot_list.append(i.solution_value())
right_femur_percentage_overshoot = right_femur_percentage_overshoot.solution_value()
right_femur_threshold = []
for i in right_femur_above_threshold:
    right_femur_threshold.append(i.solution_value())

CTV_overshoot_list = []
CTV_undershoot_list = []
for i in range(CTV_voxel_count):
    CTV_overshoot_list.append(CTV_overshoot[i].solution_value())
    CTV_undershoot_list.append(CTV_undershoot[i].solution_value())


# Dose Matrix Visualization
# solution_matrix = np.zeros((20, 20))
# solution_dose_matrix = np.reshape(voxel_doses, (20,20))
# for i in range(20):
#     for j in range(20):
#             solution_matrix[i][j] = solution_dose_matrix[i][j].solution_value()
# pd.DataFrame(solution_matrix).round(decimals=2)

# Voxel Doses
# print("Unspecified:")
# for voxel_ID in unspecified_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Rectum:")
# for voxel_ID in rectum_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Bladder:")
# for voxel_ID in bladder_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Left Femur:")
# for voxel_ID in left_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("Right Femur:")
# for voxel_ID in right_femur_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")
# print("CTV:")
# for voxel_ID in CTV_voxels:
#     print("Voxel_"+str(voxel_ID), voxel_doses[voxel_ID-1].solution_value())
# print("\n")


# Beamlet Intensities
for i in range(len(beamlet_intensities)):
    print("Beamlet_"+str(i+1), beamlet_intensities[i].solution_value())
